# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
import pprint

from leap_ec.individual import Individual

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[1, 0, 1, 1, 1], [0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 1, 1, 1, 0]]
1 [[1, 0, 1, 0, 0]]
2 [[0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]
3 [[0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]
4 [[0, 0, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1], [1, 1, 0, 0, 0]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[3, 6]]
1 [[2, 4]]
2 [[0, 4], [1, 1], [1, 6]]
3 [[0, 5], [0, 2], [0, 6], [1, 7]]
4 [[0, 0], [3, 5], [1, 5], [0, 7], [3, 0]]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[1, 0], [1, 0]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pprint.pformat(s, indent=2))

0 [ [0.1599554258520426, -0.25128509171491786, -3.0601276376534825],
  [1.7104140266487127, -0.3245330430045128, -6.712495749608738],
  [-2.354195681567262, 0.49370472366475826, 9.346940995571465]]
1 [ [-1.4903059436408932, -0.8922492759452998, 1.8277551242250407],
  [5.040471078310923, -0.5796246595246342, -2.952836298294197]]
2 [ [3.850950351591556, -0.3644893360593733, 8.659795499983066],
  [3.711852347484, 0.49682065654737717, -3.186939707448648]]
3 [ [-0.1683112851581372, 0.2187850097616708, -7.00376038541185],
  [-3.2775124793504498, 0.8636697222276843, 5.725397073895422]]
4 [[4.2895653633405795, -0.6749388437672366, -5.490606071862403]]
5 [ [2.8873211142770385, -0.5135169942816131, 8.207739497802784],
  [2.76238767510728, -0.39467840870766646, 2.567767322286631],
  [1.851602055789547, 0.010409694810307402, -3.189066064806587],
  [2.781463094551899, 0.6030862807700004, -0.9846779947925164],
  [1.0114778085880483, 0.8691159207652073, -2.718013842840299]]


Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [7]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
mutated: [[0.0, 0.14899638188431782], [1.0, 1.0], [-1.0, 0.0]]
